## 필요한 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [10]:
%matplotlib inline 

In [ ]:
##그림도 interpreter 형식으로 jupyter에만 적용, pycharm은 cell 단위가 아니므로 매직명령어 사용 x

## example 데이터 불러오기

In [5]:
ex = pd.read_csv("./input/example.csv")

In [7]:
# 데이터 확인
ex.head()

,번호,나이,키,몸무게,최종학력,연봉,다음기수
0,1,21,170,70,고등학교,3000,O
1,2,24,175,75,대학교,3200,X
2,3,23,180,80,고등학교,3400,X
3,4,22,185,85,대학교,2800,O
4,5,25,160,60,대학원,4000,O


## 데이터 프레임 정보 확인하기

In [12]:
ex.info()
# 용량이 클 때 type 변경 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
번호      8 non-null int64
나이      8 non-null int64
키       8 non-null int64
몸무게     8 non-null int64
최종학력    8 non-null object
연봉      8 non-null int64
다음기수    8 non-null object
dtypes: int64(5), object(2)
memory usage: 576.0+ bytes


# One Hot Encoding

## (1) pandas 모듈 사용

In [38]:
# get_dummies() ... 가능하면 함수 하나로 하지말고 직접 구현할 수 있어야 함
# 10010001 이런 식
temp = pd.get_dummies(ex['최종학력'])

In [39]:
# 최종 데이터 프레임 만들기
final = pd.concat([ex, temp], axis=1) # 열로 이어붙여야 하므로 1, 행은 1

In [40]:
final.head()

,번호,나이,키,몸무게,최종학력,연봉,다음기수,고등학교,대학교,대학원
0,1,21,170,70,고등학교,3000,O,1,0,0
1,2,24,175,75,대학교,3200,X,0,1,0
2,3,23,180,80,고등학교,3400,X,1,0,0
3,4,22,185,85,대학교,2800,O,0,1,0
4,5,25,160,60,대학원,4000,O,0,0,1


In [41]:
# 필요없는 컬럼 지우기
final2 = final.drop(['최종학력'], axis = 1)

In [42]:
# 최종 데이터 프레임 확인하기
final2.head()

,번호,나이,키,몸무게,연봉,다음기수,고등학교,대학교,대학원
0,1,21,170,70,3000,O,1,0,0
1,2,24,175,75,3200,X,0,1,0
2,3,23,180,80,3400,X,1,0,0
3,4,22,185,85,2800,O,0,1,0
4,5,25,160,60,4000,O,0,0,1


In [45]:
final.drop(['최종학력'], axis = 1, inplace = True) # 원본 변경, 신중히 할 것

In [46]:
final.head()

,번호,나이,키,몸무게,연봉,다음기수,고등학교,대학교,대학원
0,1,21,170,70,3000,O,1,0,0
1,2,24,175,75,3200,X,0,1,0
2,3,23,180,80,3400,X,1,0,0
3,4,22,185,85,2800,O,0,1,0
4,5,25,160,60,4000,O,0,0,1


In [44]:
# 한번에 처리
# pd.get_dummies(data, columns, drop_first)
# data : 데이터 프레임
# columns : one-hot encoding처리하고 싶은 column
# drop_first : True of False

pd.get_dummies(ex, columns = ['최종학력'], drop_first = False)
# default : false

,번호,나이,키,몸무게,연봉,다음기수,최종학력_고등학교,최종학력_대학교,최종학력_대학원
0,1,21,170,70,3000,O,1,0,0
1,2,24,175,75,3200,X,0,1,0
2,3,23,180,80,3400,X,1,0,0
3,4,22,185,85,2800,O,0,1,0
4,5,25,160,60,4000,O,0,0,1
5,6,23,155,55,3600,X,0,1,0
6,7,22,158,58,3800,O,0,1,0
7,8,26,166,66,3700,O,1,0,0


## (2) sklearn 모듈 사용

In [58]:
# 머신러닝의 집합체 - 모델에 넣기전 모든 전처리 과정
# 모델에 넣는 것은 parameter만 수정하면 되는 것.. 전처리 매우 중요
# 라이브러리 불러오기
from sklearn.preprocessing import OneHotEncoder # import 뒤만 불러오겠다
# 다불러 오면 안쓸 모듈도 불러오게 됨...

# numpy를 사용해 배열로 변경 - 머신러닝에서는 데이터 프레임으로 진행해도 되지만 딥러닝에서는 꼭 array로 바꿨다가 데이터 프레임으로 변경
temp = np.array(ex['최종학력'])
temp # 1행 8열... one hot encoding 불가 -> 1열로 

# OneHotEncoder를 넣기 위해 배열의 shape변경
temp2 = temp.reshape(-1,1) # 원하는대로 size 변경 - 후자가 1이면 8행 1열로, 후자가 2면 4행 2열로

# OneHotEncoder 생성
onehot = OneHotEncoder()

# OneHotEncoder '학습'시키기
onehot.fit(temp2)

# OneHotEncoder '적용'시키기
onehot.transform(temp2).toarray()

# concat, drop 없이 가능, get_dummies에 매몰되지 말 것

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

## Label Encoding 사용

In [113]:
# 라이브러리 불러오기
# column이 하나, 각각의 column에 이름을 붙여줌
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 생성
label = LabelEncoder()

# LabelEncoder 적용
ex['최종학력'] # Series 형태 - 인식 불가
label_f=ex[['최종학력']].apply(label.fit_transform) # 대괄호 한 번 더 - data frame

In [109]:
ex.head()

,번호,나이,키,몸무게,최종학력,연봉,다음기수
0,1,21,170,70,고등학교,3000,O
1,2,24,175,75,대학교,3200,X
2,3,23,180,80,고등학교,3400,X
3,4,22,185,85,대학교,2800,O
4,5,25,160,60,대학원,4000,O


In [114]:
# 최종 데이터 프레임 만들기
final_label = pd.concat([ex, label_f], axis=1)#drop(['최종학력'], axis = 1, inplace = True)
# ex.drop(['최종학력'])

In [115]:
final_label.head()

,번호,나이,키,몸무게,최종학력,연봉,다음기수,최종학력
0,1,21,170,70,고등학교,3000,O,0
1,2,24,175,75,대학교,3200,X,1
2,3,23,180,80,고등학교,3400,X,0
3,4,22,185,85,대학교,2800,O,1
4,5,25,160,60,대학원,4000,O,2


In [118]:
final_label = pd.concat([ex.drop(['최종학력'], axis=1), label_f], axis=1)#drop(['최종학력'], axis = 1, inplace = True)

In [119]:
final_label.head()

,번호,나이,키,몸무게,연봉,다음기수,최종학력
0,1,21,170,70,3000,O,0
1,2,24,175,75,3200,X,1
2,3,23,180,80,3400,X,0
3,4,22,185,85,2800,O,1
4,5,25,160,60,4000,O,2


In [69]:
label_f.head()

,최종학력
0,0
1,1
2,0
3,1
4,2


# K-NN

## 데이터 불러오기

### 독립변수 (나이, 키, 연봉) -알고리즘 (function)-> 
### clustering : 새로운 데이터가 주어졌을 때 기존의 분류에서 가장 가까운 군집에 넣어줌
### 가까운 k개를 구분지어줌
### model 자체에서 parameter 조정 : tuning, hyper-parameter
### 어떤 데이터에서는 어떻게 분류되는 지 각각 구분지어봐야 함 : k 지정에 따라 다름
### hyper parameter가 k 하나임

### 거리 측정 방법
### Euclidian : 피타고라스 공식, 빗변 길이 - 차원이 많아지면 계속 변수벡터 늘려가면서 빼고 루트
### Manhattan : 절댓값

In [71]:
wine = pd.read_csv('./input/wine.csv')

In [72]:
wine.head() # 변수 의미 알아야

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


## grade라는 column name으로 quality의 값이 3 ~ 6이면 Good / 7 ~ 9이면 Best로 할당

In [84]:
# value_count
wine['quality'].value_counts() # .sort(increasing = True)

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [88]:
# for문 연산속도 제일 느림
wine['grade']=['Good' if x<=6 else 'Best' for x in wine['quality']] # 6부터 담김, 없던 column name 쓰면 접근 아니고 새로 생성
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,grade
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,Good
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,Good
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,Good
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,Good
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,Good


## wine 요약통계량 확인

In [90]:
wine.describe() # 명목형은 적용 불가 - R은 describe 했을 때 개수도 보여줌 ...
# residual.sugar ... std가 크므로 분포가 평균에서 스펙트럼이 좀 넓다 \
# std가 작으면 다 고만고만 하니까 구분이 잘 안 됨... 너무 커도 극적으로 가므로 좋지 않다.
# 단위 차이 떄문에 별로 크게 나지 않을 수도 ex. ratio... - domain 지식 활용
# 연속형일 때는 반드시 단위를 통일, 맞춰줘야 연산할 수 있음
# 다 일일히 맞출 수는 없으므로 표준화를 통해 std 연산

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


## 표준화

In [106]:
from sklearn.preprocessing import StandardScaler

# scaler 생성
standard_scaler = StandardScaler()

# scaler 학습 : 연속형만 들어와야 함
# 표준 정규 분포로 표현 가능해야 - 퍙균에서 얼마나 떨어졌는가
# 0.27-1.27 / 1
standard_scaler.fit(x_data)

# scaler 적용
temp = standard_scaler.transform(x_data)

# 최종 데이터프레임 만들기
pd.DataFrame(temp, columns = x_data.columns)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol
0,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152
1,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276
2,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667
3,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203
4,0.409125,-0.478657,-0.117278,0.415768,0.559727,0.687541,1.121091,0.525855,0.011480,-0.787342,-0.499203
...,...,...,...,...,...,...,...,...,...,...,...
4893,-0.776015,-0.677101,-0.365197,-0.944765,-0.310008,-0.664970,-1.091000,-0.965483,0.541334,0.088973,0.557282
4894,-0.301959,0.414339,0.213280,0.317179,0.056196,1.275590,0.697499,0.291789,-0.253446,-0.261553,-0.743008
4895,-0.420473,-0.379435,-1.191592,-1.023637,-0.218457,-0.312141,-0.643875,-0.497350,-1.313153,-0.261553,-0.905544
4896,-1.605613,0.116674,-0.282557,-1.043355,-1.088192,-0.900190,-0.667408,-1.784717,1.004955,-0.962605,1.857572


## 종속변수, 독립변수 분할
### target, 요인

In [95]:
y_data = wine['grade']
# quality (답)를 넣어놓고 분석하면 당연 99%...
x_data = wine.drop(['quality', 'grade'], axis = 1)

## train, test 분할
### 지도학습, 주로 7:3/8:2로 분리, 8로 모델을 만들고 2로 test, 모델 성능 검정
### cross validation

In [122]:
from sklearn.model_selection import train_test_split # 마구잡이로 뽑음

X_train, X_test, Y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 321)

In [125]:
print(X_train.shape)
print(X_test.shape)

(3918, 11)
(980, 11)


In [124]:
# 데이터 크기 확인
4898*0.8

3918.4

## K-NN 모델 적용

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# knn 모델 생성
# n_neighbors : k의 개수
# p : 1(맨해튼), 2(유클리디안)
# weights : uniform(동일), distance(거리기반 가중치)
# n_jobs : -1을 사용하면 현재 컴퓨터내 모든 process 사용


In [ ]:
# knn 모델 학습


In [ ]:
# 확률 출력


In [ ]:
# 학습한 모델을 바탕으로 class 예측


In [ ]:
# 정확도 출력


In [ ]:
# k값에 따른 정확도 확인


In [ ]:
# k값에 따른 정확도 시각화


### 지도 학습 : 답이 있어서 채점 가능, 정확도, 민감도, 특이도 측정
### 모델을 만들고 test, 타겟이 존재 - 답에서 얼마나 벗어났는지
### xg boost
### 분류  - KNN (범주형도 가능) Naive Bayes (예측 불가) Decision Tree 예측 가능
### 예측 - linear Regression (성능 안좋아서 논문용...)

### 비지도 학습 : 비슷한 애들끼리 학습, 그 자체로 결과, 정답 x
### clustering - 마케팅 타겟집단 분류 : k-means, DBScan
### 데이터 공모전 - 모델 성능 예측 : 지도 학습, 사업 아이디어 : 대부분 군집화
### 연봉.. 연속형
### AB Test
### 컴퓨터는 다 숫자로 detect -문자로 들어가있는 것은 error : 
### one-hot encoding : 문자를 숫자로 구분해줌 - 해당 되는 곳만 1
### multi class classification - 사람, 동물 구분 ...
### 차원 축소 - 최대한 column을 줄여줌 ex. 고등학교 대학교 대학원
### 고등학교 = 대학교 0 대학원 0
### label encoding : 0 1 형태 x, 0 1 2 ... 종속변수는 모델에 넣지 않음, (모델에 넣는 것은 독립변수 - label encoding x)
### 학력 구분을 위함이지 학력의 차이를 의미하지 않음 - target 변수 변경